# Mechanisms-of-Action-Prediction-baseline-lightGBM

## debug

In [43]:
debug = True

## import libraries

In [44]:
import pandas as pd
import numpy as np
import copy
from sklearn.metrics import log_loss

import lightgbm as lgb

## load data

In [45]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
test_features = pd.read_csv("../input/lish-moa/test_features.csv")
submission = pd.read_csv("../input/lish-moa/sample_submission.csv")

## preprocess

In [46]:
prep_df = pd.concat([train_features, test_features])

In [47]:
import category_encoders as ce

object_cols = ["cp_type", "cp_dose"]
ce_oe = ce.OrdinalEncoder(cols=object_cols, handle_unknown='impute')
prep_df = ce_oe.fit_transform(prep_df)
train_features = prep_df.iloc[:len(train_features),:]
test_features = prep_df.iloc[len(train_features):,:]

## train models and prediction

In [48]:
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
FOLD_NUM = 5
kf = KFold(n_splits=5, shuffle=True, random_state=0)
target_cols = [c for c in train_targets_scored.columns if c != "sig_id"]
oof = train_targets_scored.copy()
num_round = 10000

In [50]:
params = {
#'boosting_type': 'gbdt',
'objective': 'binary',
#'lambda_l1': 0.001, 
#'lambda_l2': 0.001,
'num_leaves': 50, 
'max_depth': 6,
#'feature_fraction': 0.4,
#'subsample': 0.4, 
#'min_child_samples': 10,
'learning_rate': 0.01,
#'num_iterations': 100, #700
'early_stopping_rounds': 100,
'random_state': 42}

In [54]:
def run_lgbm(target_col: str):
    
    X_train = train_features.drop(["sig_id"], axis=1)
    y_train = train_targets_scored[target_col]
    X_test = test_features.drop(["sig_id"], axis=1)

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))

    for fold_id, (train_index, valid_index) in enumerate(kf.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=object_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=object_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
        
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

In [55]:
for target_col in target_cols:
    _oof, _preds = run_lgbm(target_col)
    oof[target_col] = _oof
    submission[target_col] = _preds

/home/kimura_sampler/anaconda3/envs/probspace_realestate/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kimura_sampler/anaconda3/envs/probspace_realestate/lib/python3.8/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.00236783	valid_1's binary_logloss: 0.0068585
[20]	training's binary_logloss: 0.00181572	valid_1's binary_logloss: 0.00688485
[30]	training's binary_logloss: 0.00150247	valid_1's binary_logloss: 0.00691576
[40]	training's binary_logloss: 0.00128277	valid_1's binary_logloss: 0.00695037
[50]	training's binary_logloss: 0.00111277	valid_1's binary_logloss: 0.00698997
[60]	training's binary_logloss: 0.000976128	valid_1's binary_logloss: 0.00702704
[70]	training's binary_logloss: 0.000863108	valid_1's binary_logloss: 0.00702334
[80]	training's binary_logloss: 0.00076729	valid_1's binary_logloss: 0.00703847
[90]	training's binary_logloss: 0.000684898	valid_1's binary_logloss: 0.00706433
[100]	training's binary_logloss: 0.000613379	valid_1's binary_logloss: 0.0070939
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00479453	valid_1's binary_logloss: 0.00681599
Training until valida

[30]	training's binary_logloss: 0.00148971	valid_1's binary_logloss: 0.0100949
[40]	training's binary_logloss: 0.0012776	valid_1's binary_logloss: 0.0101784
[50]	training's binary_logloss: 0.00111616	valid_1's binary_logloss: 0.0102491
[60]	training's binary_logloss: 0.000984331	valid_1's binary_logloss: 0.010325
[70]	training's binary_logloss: 0.00087552	valid_1's binary_logloss: 0.0103921
[80]	training's binary_logloss: 0.000782081	valid_1's binary_logloss: 0.0104816
[90]	training's binary_logloss: 0.000700959	valid_1's binary_logloss: 0.0105739
[100]	training's binary_logloss: 0.000629586	valid_1's binary_logloss: 0.0106743
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.0049334	valid_1's binary_logloss: 0.00992439
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0030805	valid_1's binary_logloss: 0.00686536
[20]	training's binary_logloss: 0.00219902	valid_1's binary_logloss: 0.00685029
[30]	training's binary_logloss:

KeyboardInterrupt: 

In [ ]:
scores = []
for target_col in target_cols:
    scores.append(log_loss(train_targets_scored[target_col], oof[target_col]))
print(np.mean(scores))

In [ ]:
if debug == False:
    submission.to_csv("submission.csv", index=False)
    submission.head()

## optuna

In [ ]:
def objective(trial):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        #'metric': 'rmse',
        'verbosity': -1,
        "seed":42,
        "learning_rate":trial.suggest_loguniform('learning_rate', 0.005, 0.03),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }
    FOLD_NUM = 5
    mskf = MultilabelStratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=42)
    scores = []
    feature_importance_df = pd.DataFrame()

    pred_cv = np.zeros(len(test.index))
    num_round = 10000

    


    for i, (tdx, vdx) in enumerate(kf.split(train_X[selected], train_y)):
        print(f'Fold : {i}')
        X_train, X_valid, y_train, y_valid = train_X[selected].iloc[tdx], train_X[selected].iloc[vdx], train_y.values[tdx], train_y.values[vdx]
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=object_cols)
        lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature=object_cols)
        model = lgb.train(params, lgb_train, num_boost_round=num_round,
                      valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                      early_stopping_rounds=100, verbose_eval=10000)

        va_pred = model.predict(X_valid)
        va_pred[va_pred<0] = 0
        score_ = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(va_pred)))
        scores.append(score_)

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# 結果の確認
print('Best trial:')
light_trial = study.best_trial

print('  Value: {}'.format(light_trial.value))

print('  Params: ')

In [ ]:
with open("lightgbmparams.txt", "w") as file:
    for key, value in light_trial.params.items():
       print('    "{}": {},'.format(key, value))
       file.write('"{}": {},'.format(key, value))